큰 파일의 경우 클라우드에 올려서 실행 시 다운로드 받도록 하는 것이 편리한데,  구글 클라우드의 경우에는 나름 간편하게 받을 수 있다.

아래 cell은 파일을 ./input아래에 다운받고(이미 파일이 있으면 skip) 압축을 해제한다.

In [0]:
# 데이터에 문제가 있어 다시 받아야 하면 아래 코드 실행

import shutil
#shutil.rmtree("./input")

In [0]:
# 서버의 구글 드라이브 파일을 다운받는다.

from google_drive_downloader import GoogleDriveDownloader as gdd
import os


#train.zip : https://drive.google.com/open?id=1ZpsImi7TtTfzG9AVn340mEqR6ie57Dqm
#test.zip  : https://drive.google.com/open?id=14okHHWLzWXTrIBranvx8sGurvToLNSNv
#csvs.zip  : https://drive.google.com/open?id=1du8AdwdHrM6fdZWloeXSMZDG6UhbmAKb
#train_crop.zip : https://drive.google.com/open?id=1jCnPqPwb1w3LnFHWIj4eoDGsfBIiRb3K
#test_crop.zip : https://drive.google.com/open?id=1RLF_4MkqK6-5uyv8gLOl-7FwkZzyWudk

def download_file_gd(file_id, fpathname, unzip=False):
    if os.path.exists(fpathname) == False:
        gdd.download_file_from_google_drive(file_id=file_id, dest_path=fpathname, unzip=unzip, showsize=True)
    else:
        print(fpathname, ": already downloaded")

files = {"1ZpsImi7TtTfzG9AVn340mEqR6ie57Dqm" : "./input/train/train.zip", 
 "14okHHWLzWXTrIBranvx8sGurvToLNSNv" : "./input/test/test.zip",
 "1du8AdwdHrM6fdZWloeXSMZDG6UhbmAKb" : "./input/csvs.zip",
 "1jCnPqPwb1w3LnFHWIj4eoDGsfBIiRb3K" : "./input/train_crop/train_crop.zip",
 "1RLF_4MkqK6-5uyv8gLOl-7FwkZzyWudk" : "./input/train_crop/test_crop.zip"}

for f in files:
    print(f, files[f])
    download_file_gd(file_id=f, fpathname=files[f], unzip=True)


# 2019 3rd ML month with KaKR

자동차 이미지 데이터셋을 이용한 자동차 차종 분류

## 목차

**1. [데이터 분석](#data_analysis)** <br>
**2. [데이터 편집](#data_manipulation)** <br>
**3. [모델 생성/훈련/제출](#modelling_training_submit)** <br>



<a id="data_analysis"></a> <br> 
# **1. 데이터 분석:**

train.csv/test.csv 파일을 Pandas DataFrame에 읽어들입니다.

In [1]:
import pandas as pd
import os

data_path = "../input"

df_train = pd.read_csv(os.path.join(data_path, "train.csv"))
df_test = pd.read_csv(os.path.join(data_path, "test.csv"))

df_class = pd.read_csv(os.path.join(data_path, "class.csv"))

In [2]:
df_train.head()

,img_file,bbox_x1,bbox_y1,bbox_x2,bbox_y2,class
0,train_00001.jpg,1,80,641,461,108
1,train_00002.jpg,57,53,293,236,71
2,train_00003.jpg,35,42,227,160,76
3,train_00004.jpg,10,21,254,165,188
4,train_00005.jpg,1,67,772,573,44


이미지 파일명과 이미지에서의 자동차 위치를 표시하는 좌표(left,top,right,bottom)값, 그리고 자동차를 식별하는 class가 있다.


In [3]:
df_test.head()

,img_file,bbox_x1,bbox_y1,bbox_x2,bbox_y2
0,test_00001.jpg,199,184,1116,809
1,test_00002.jpg,55,61,480,257
2,test_00003.jpg,42,123,602,413
3,test_00004.jpg,13,8,619,393
4,test_00005.jpg,8,9,209,93


test.csv는 class가 없는 것을 제외하고 train과 동일하다.

In [4]:
df_class.head()

,id,name
0,1,AM General Hummer SUV 2000
1,2,Acura RL Sedan 2012
2,3,Acura TL Sedan 2012
3,4,Acura TL Type-S 2008
4,5,Acura TSX Sedan 2012


(id,name)쌍으로 된 자동차 class와 이름 데이터가 들어있다.

이미지들의 width/height의 min/max 값과 분포를 확인합니다.

In [5]:
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import numpy as np
import os


widths = []
heights = []

for idx, row in df_train.iterrows():    
    im = Image.open(os.path.join(os.path.join(data_path, "train"), row['img_file']))
    w, h = im.size
    widths.append(w)
    heights.append(h)
    
widths = np.array(widths)
heights = np.array(heights)
print("width  min/max/mean : ", np.min(widths), np.max(widths), np.mean(widths))
print("height min/max/mean : ", np.min(heights), np.max(heights), np.mean(heights))

bins = [x for x in range(-1, 6000, 200)]

import matplotlib.pyplot as plt
import seaborn as sns
fig, ax = plt.subplots(2, 2, figsize=(20, 20))

# draw as hist:
ax[0][0].hist(widths, bins=bins)
ax[0][1].hist(heights, bins=bins)

# draw as kde:
sns.kdeplot(np.array(widths), ax=ax[1][0])
sns.kdeplot(np.array(heights), ax=ax[1][1])
    


width  min/max/mean :  78 5616 698.2644644644645
height min/max/mean :  57 3744 481.853953953954


이미지 몇 장을 box를 그려서 출력해 봅니다.

In [6]:
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

n = 5

fix, ax = plt.subplots(n, 1, figsize = (10, 40))
axidx = 0

df_sample = df_train.sample(n)
for idx, row in df_sample.iterrows():
    im = Image.open(os.path.join(os.path.join(data_path, "train"), row['img_file']))
    
    # 이미지 내에 박스를 그린다.
    draw = ImageDraw.Draw(im)
    draw.rectangle((row['bbox_x1'], row['bbox_y1'], row['bbox_x2'], row['bbox_y2']), outline='blue')
    
    print("bbox:", row['bbox_x1'], row['bbox_y1'], row['bbox_x2'], row['bbox_y2'])
    ax[axidx].imshow(im)
    axidx += 1


bbox: 31 31 1543 1018
bbox: 47 35 590 403
bbox: 3 30 295 192
bbox: 109 196 1483 1087
bbox: 115 104 783 349


사이즈는 resize가 어렵지 않지만 ratio는 그렇지 않습니다.
ratio의 분포를 학인할 필요가 있어 보입니다.

In [7]:
def checkratio(df):
    df['ratio'] = (df['bbox_x2'] - df['bbox_x1']) / (df['bbox_y2'] - df['bbox_y1'])
    return df

df_merged = df_train.append(df_test, ignore_index=True)
df_tmp = checkratio(df_merged)
print(df_tmp.head())
print(df_tmp['ratio'].head(10))
df_tmp['ratio'].plot.kde()
print("ratio.mean():", df_tmp['ratio'].mean())

#default_ratio = df_tmp['ratio'].mean()

default_ratio = 1.0

   bbox_x1  bbox_x2  bbox_y1  bbox_y2  class         img_file     ratio
0        1      641       80      461  108.0  train_00001.jpg  1.679790
1       57      293       53      236   71.0  train_00002.jpg  1.289617
2       35      227       42      160   76.0  train_00003.jpg  1.627119
3       10      254       21      165  188.0  train_00004.jpg  1.694444
4        1      772       67      573   44.0  train_00005.jpg  1.523715
0    1.679790
1    1.289617
2    1.627119
3    1.694444
4    1.523715
5    1.257919
6    1.523077
7    1.842262
8    2.111111
9    1.957447
Name: ratio, dtype: float64
ratio.mean(): 1.945869718450243


/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


crop된 이미지를 모델의 입력으로 사용하기 위해서는 이미지의 크기를 일정한 크기로 resize해야 합니다.

그러나 이미지를 그대로 resize하면 이미지의 ratio가 변형되어 데이터가 손상되므로 resize전에 먼저 이미지의 좌/우 혹은 위/아래에 padding을 추가하여 동일한 ratio로 맞춰 주어야 합니다.

In [8]:
image_width, image_height = int(default_ratio * 200), int(200)

def get_fixed_img(filename, area, ratio, output_size):
    debug = False
    im = Image.open(filename)
    cropIm = im.crop(area)
    
    if debug:
        print("crop w/h=", cropIm.width, cropIm.height)
    
    w, h = cropIm.width, cropIm.height
    # w : h = w/h ratio : 1
    fixedW, fixedH = w, h
    if w/h >= ratio:
        fixedH = w / ratio
    else:
        fixedW = h * ratio
    fixedW, fixedH = int(fixedW), int(fixedH)

    if debug:
        print("fixed w/h=", fixedW, fixedH, "ratio=", fixedW/fixedH)
    
    newIm = Image.new("RGB", (fixedW, fixedH))
    newIm.paste(cropIm, ((fixedW - w)//2, (fixedH - h)//2))
    
    newIm = newIm.resize(output_size, resample=Image.NEAREST)
    return newIm

def img_from_row(row, path, ratio, w, h):
    filepath = os.path.join(path, row['img_file'])
    area = (row['bbox_x1'], row['bbox_y1'], row['bbox_x2'], row['bbox_y2'])
    return get_fixed_img(filepath, area, ratio, (w, h))    

def make_crop_img(df, dirpath):
    for idx, row in df.iterrows():
        im = img_from_row(row, dirpath, default_ratio, image_width, image_height)
        dirpath_crop = dirpath + "_crop"
        if os.path.exists(dirpath_crop) == False:
            os.mkdir(dirpath_crop)        
        new_path = os.path.join(dirpath_crop, row['img_file'])
        im.save(new_path)        

make_crop_img(df_train, os.path.join(data_path, "train"))
make_crop_img(df_test, os.path.join(data_path, "test"))
    

In [9]:
a,b = (1,2)

In [10]:
print(a,b)

1 2
